# Silicone
### Erica Simon, 02/13/24
## Purpose: use the Silicone infilling tool to fill missing emissions data for certain species
Credit: 
- Lamboll, R. D., Nicholls, Z. R. J., Kikstra, J. S., Meinshausen, M., & Rogelj, J. (2020). Silicone v1.0.0: an open-source Python package for inferring missing emissions data for climate change research. *Geoscientific Model Development, 13(11),* 5259–5275. https://doi.org/10.5194/gmd-13-5259-2020
- Meinshausen, M., Lewis, J., McGlade, C. et al. Realization of Paris Agreement pledges may limit warming just below 2 °C. *Nature* 604, 304–309 (2022). https://doi.org/10.1038/s41586-022-04553-z
    - code at https://github.com/climate-resource/ndc-realisations-2021



## Reason for Archiving 
Ultimately, Silicone was not used for the project. Instead, missing species were infilled using trends from the F-gases basket provided or, for non-F-gases, using the CO2 trend.

In [5]:
import pandas as pd
import numpy as np
import pyam

import silicone.multiple_infillers as mi
import silicone.database_crunchers as cr

In [6]:
import silicone.database_crunchers
from silicone.time_projectors import ExtendLatestTimeQuantile
import scmdata
import scmdata.database
import matplotlib.pyplot as plt

from tqdm.autonotebook import tqdm

In [7]:
future_df = pd.read_csv('../outputs/GCAM_cleaned.csv')
hist_df = pd.read_csv('../outputs/hist_emis_cleaned.csv')

df_to_infill = pyam.IamDataFrame(future_df)
df = pyam.IamDataFrame(hist_df)

/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/utils.py:316: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.set_index(index + REQUIRED_COLS + extra_cols)
/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/utils.py:316: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.set_index(index + REQUIRED_COLS + extra_cols)


In [8]:
missing_vars = np.setdiff1d(hist_df['Variable'].unique(), future_df['Variable'].unique())

In [9]:
lead = ['Emissions|CO2 FFI']
variables_of_interest = ['Emissions|C3F8']
years_list = list(range(2022, 2101))

In [10]:
unavailable_variables = [
        variab for variab in variables_of_interest if variab not in df.variable
    ]


In [11]:
unavailable_variables

[]

In [12]:
df

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : Historical (1)
 * scenario : GCP+CEDS+PRIMAP+GFED (1)
Timeseries data coordinates:
   region   : World (1)
   variable : Emissions|BC, Emissions|C2F6, Emissions|C3F8, ... Emissions|c-C4F8 (51)
   unit     : Gt CO2/yr, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt N2O/yr, ... kt cC4F8/yr (50)
   year     : 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1757, ... 2022 (273)

In [13]:
df_to_infill

<class 'pyam.core.IamDataFrame'>
Index:
 * model    : GCAM 6.0 NGFS (1)
 * scenario : Below 2 C, Current Policies, Delayed transition, ... Net Zero 2050 (7)
Timeseries data coordinates:
   region   : World (1)
   variable : Emissions|BC, Emissions|C2F6, Emissions|C3F8, ... Emissions|c-C4F8 (51)
   unit     : Gt CO2/yr, Mt BC/yr, Mt CH4/yr, Mt CO/yr, Mt N2O/yr, ... kt cC4F8/yr (50)
   year     : 2020, 2021, 2022, 2023, 2024, 2025, 2026, 2027, ... 2100 (81)

In [14]:
df_infilled = mi.infill_all_required_variables(
    df_to_infill,
    df,
    variable_leaders=lead,
    required_variables_list=variables_of_interest, # If None, would infill a default list
    cruncher=cr.QuantileRollingWindows,
    output_timesteps=years_list,
    infilled_data_prefix=None,
    to_fill_old_prefix=None,
    check_data_returned=False,
)

Filling required variables:   0%|          | 0/1 [00:00<?, ?it/s]/opt/miniconda/envs/risk/lib/python3.10/site-packages/silicone/database_crunchers/quantile_rolling_windows.py:182: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  wide_db = wide_db.applymap(lambda x: np.nan if isinstance(x, str) else x)
/opt/miniconda/envs/risk/lib/python3.10/site-packages/pyam/core.py:895: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  self.meta[name] = meta[name].combine_first(self.meta[name])
Filling required variables: 100%|██████████| 1/1 [00:00<00:00, 10.07it/s]


In [15]:
df_infilled.filter(variable=variables_of_interest[0]).timeseries()

2022  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.392255   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.386953   
              NDCs               World  Emissions|C3F8 kt C3F8/yr  0.398573   
              Net Zero 2050      World  Emissions|C3F8 kt C3F8/yr  0.389460   

                                                                       2023  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.398346   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.407678   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.407678   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.407679   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.390393   
              NDCs               World  Emissions|C3F8 kt C3F8/yr  0.407823   
              Net Zero 2050      World  Emissions|C3F8 kt C3F8/yr  0.394153   

                                                                       2024  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.404437   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.393833   
              NDCs               World  Emissions|C3F8 kt C3F8/yr  0.417073   
              Net Zero 2050      World  Emissions|C3F8 kt C3F8/yr  0.398846   

                                                                       2025  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.410527   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.426082   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.426081   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.426082   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.397273   
              NDCs               World  Emissions|C3F8 kt C3F8/yr  0.426323   
              Net Zero 2050      World  Emissions|C3F8 kt C3F8/yr  0.403539   

                                                                       2026  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.412879   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.441407   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.441406   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.441407   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.392539   
              NDCs               World  Emissions|C3F8 kt C3F8/yr  0.439007   
              Net Zero 2050      World  Emissions|C3F8 kt C3F8/yr  0.399256   

                                                                       2027  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.415230   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.456732   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.456731   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.456732   
              Low demand         Worl

In [16]:
df_to_infill.filter(variable=variables_of_interest[0]).timeseries().head()

2020  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.380074   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.380074   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.380074   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.380074   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.380074   

                                                                       2021  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.386165   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.389275   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.389275   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.389275   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.383514   

                                                                       2022  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.392255   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.398477   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.386953   

                                                                       2023  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.398346   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.407678   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.407678   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.407679   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.390393   

                                                                       2024  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.404437   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.416880   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.393833   

                                                                       2025  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.410527   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.426082   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.426081   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.426082   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.397273   

                                                                       2026  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          World  Emissions|C3F8 kt C3F8/yr  0.412879   
              Current Policies   World  Emissions|C3F8 kt C3F8/yr  0.441407   
              Delayed transition World  Emissions|C3F8 kt C3F8/yr  0.441406   
              Fragmented World   World  Emissions|C3F8 kt C3F8/yr  0.441407   
              Low demand         World  Emissions|C3F8 kt C3F8/yr  0.392539   

                                                                       2027  \
model         scenario           region variable       unit                   
GCAM 6.0 NGFS Below 2 C          Wo

In [17]:
def extend_timeseries(infilling_database, scenario, lead='Emissions|CO2 FFI', smoothing=0):
    cruncher = silicone.time_projectors.ExtendLatestTimeQuantile(
        # infilling_database.filter(year=range(2022, 2101, 5)).to_iamdataframe()
        infilling_database.filter(year=range(2022, 2101, 5))
        
    )
    
    filler = cruncher.derive_relationship(lead, smoothing=smoothing)

    # scenario["variable"] = lead
    # extended_scenario = filler(scenario.to_iamdataframe())
    extended_scenario = filler(scenario)
    extended_scenario = scmdata.ScmRun(scenario.append(extended_scenario)).resample(
        "AS"
    )
    extended_scenario["stage"] = "extended"
    return extended_scenario


In [18]:
selected_scenarios = df_to_infill.filter(
    year=range(2022, 2100 + 1)
)

In [19]:
extended_scenario_all = scmdata.run_append(
    [
        extend_timeseries(df_to_infill, selected_scenarios)
        # for p in tqdm(pathways)
    ]
)
# extended_scenario_2050 = extended_scenario_all.filter(pathway_id=SELECTED_PATHWAY)

ValueError: The infiller database does not extend in time past the target database, so no infilling can occur.